In [2]:
import json
import os
import sys
from typing import Any

sys.path.append(os.path.abspath(".."))

from eda.models import Generation
from eda.parsing import Conversations, Participants
from eda.utils import FOLDER_DIR

DATA_PATH = FOLDER_DIR / "data"

def export_json_data(name: str, data: dict[str, Any]):
    with DATA_PATH.joinpath(name).open("w") as fp:
        json.dump(data, fp, indent=4, ensure_ascii=False)


def create_json_data() -> dict[str, Any]:
    return {"metadata": {}, "data": {}}

participants = Participants()
conversations = Conversations(participants)

### Question 1

In [3]:
conversations.read_all(parallel=True, load_sentiments=True)

In [4]:
from collections import Counter
from collections.abc import Iterable

from eda.models import ConversationLine
from eda.sentiments import SentimentType
from eda.utils import round_precise


def sentiments_from_lines(
    lines: Iterable[ConversationLine], exclude_true_neutrals: bool = False
) -> tuple[list[str], list[float]]:
    sentiments = Counter()

    for line in lines:
        if not line.sentiments.has_loaded_scores():
            continue
        if not line.sentiments.has_scores():
            continue
        if exclude_true_neutrals and line.sentiments.neutral == 1.0:
            continue
        sentiments += line.sentiments.score_counts

    total_sum = sum(sentiments.values())
    names, scores = zip(
        *(
            (
                SentimentType(sentiment_name).display_name,
                round_precise(score / total_sum * 100, 2),
            )
            for sentiment_name, score in sentiments.items()
        )
    )
    return list(names), list(scores)


def sentiment_percentages() -> dict[str, dict[str, Any]]:
    lines_by_generation = Generation.create_mapping()
    for conversation in conversations:
        for line in conversation.lines:
            generation = line.participant.generation
            lines_by_generation[generation].append(line)

    sentiment_names = None
    generation_names = []
    data = {}

    for generation, lines in lines_by_generation.items():
        generation_names.append(generation.name)
        sentiment_names, scores = sentiments_from_lines(
            lines, exclude_true_neutrals=True
        )

        total = sum(scores)
        proportions = [
            (sentiment_name, score / total * 100)
            for sentiment_name, score in zip(sentiment_names, scores)
        ]
        
        data[generation.name] = dict(proportions)

    return data

data = create_json_data()
data["metadata"]["title"] = "Sentiment percentages per generation"
data["data"] = sentiment_percentages()
export_json_data("sentiment_percentages.json", data)

In [4]:
conversations.read_all(parallel=True, load_prosodic=True)

In [ ]:

from collections import defaultdict
from collections.abc import Generator

from eda.models import Participant


def human_name_from_snake_case(name: str) -> str:
    return " ".join(name.split("_")).capitalize()


def filter_prosodic_attributes(
    line: ConversationLine,
) -> Generator[tuple[str, str]]:
    yield from (
        (name, value) for name, value in vars(line).items() 
        if name.endswith("phrases")
        and name != "overlapping_phrases"
    )


def prosodic_frequencies(participant: Participant) -> dict[str, float]:
    participant_lines = conversations.participant_lines(participant)
    participant_data = defaultdict(int)
    n_lines = 0
    for line in participant_lines:
        for name, value in filter_prosodic_attributes(line):
            participant_data[name] += len(value)
        n_lines += 1

    norm_participant_data = {
        key: value / n_lines for key, value in participant_data.items()
    }
    return norm_participant_data


def prosodic_counts_by_generation(data: dict[Generation, Any]) -> dict[str, dict[str, float]]:
    for participant in participants:
        generation = participant.generation
        data[generation].append(prosodic_frequencies(participant))

    result = {}
    for generation, participants_frequencies in data.items():
        total_generation_counts = Counter()
        for participant_frequencies in participants_frequencies:
            total_generation_counts += Counter(participant_frequencies)

        result[generation.name] = {
            human_name_from_snake_case(prosodic_feature): count
            / len(participants_frequencies)
            for prosodic_feature, count in total_generation_counts.items()
        }
    return result


generation_map = Generation.create_mapping()
counts = prosodic_counts_by_generation(generation_map)
data = create_json_data()
data["metadata"]["title"] = "Average prosodic feature frequency per line by generation"
data["data"] = counts
export_json_data("prosodic_features.json", data)

### Question 2

In [ ]:
conversations.read_all(parallel=True, load_tagged=True)

In [17]:
from dataclasses import dataclass
from typing import Optional

from eda.language import TaggedText

TOP_N_LEMMAS = 20
ALLOWED_POS_VALUES = None
PER_WORDS = 1000

@dataclass(frozen=True)
class PosTaggedLemma:
    lemma: str
    pos_name: str


@dataclass(frozen=True)
class GenerationLemmaInfo:
    pt_lemmas: list[PosTaggedLemma]
    lemma_counts: list[int]
    n_total_words: int


def top_lemmas_by_generation(
    conversations,
    *,
    top_n: int = 10,
    min_lemma_length: int = 3,
    allowed_pos_values: Optional[set] = None,
) -> dict[Generation, GenerationLemmaInfo]:
    text_by_generation: dict[Generation, list[TaggedText]] = Generation.create_mapping()

    for conversation in conversations:
        for line in conversation:
            generation = line.participant.generation
            text_by_generation[generation].extend(line.tagged)

    lemmas_by_generation = {}
    for generation, words in text_by_generation.items():
        words_by_lemma = defaultdict(list)
        for word in words:
            if len(word.lemma) < min_lemma_length:
                continue
            if allowed_pos_values is not None and word.pos not in allowed_pos_values:
                continue
            pt_lemma = PosTaggedLemma(word.lemma, word.pos_name)
            words_by_lemma[pt_lemma].append(word)

        counts = Counter({
            pt_lemma: len(group) for pt_lemma, group in words_by_lemma.items()
        })

        lemmas, frequencies = zip(*counts.most_common(top_n))
        lemmas_by_generation[generation] = GenerationLemmaInfo(
            list(lemmas), list(frequencies), len(words)
        )

    return lemmas_by_generation

def important_lemmas() -> dict[str, Any]:
    top_lemmas = top_lemmas_by_generation(
        conversations, top_n=TOP_N_LEMMAS, allowed_pos_values=ALLOWED_POS_VALUES
    )

    result = {key.name: {} for key in Generation.create_mapping()}
    for generation, info in top_lemmas.items():
        pt_lemmas = info.pt_lemmas
        counts = info.lemma_counts
        gen_name = generation.name
        for ptl, count in zip(pt_lemmas, counts):
            lemma_frequency = round_precise((count / info.n_total_words) * PER_WORDS, 2)
            result[gen_name][ptl.lemma] = lemma_frequency
    return result


data = create_json_data()
data["metadata"]["title"] = "Top lemmas by generation"
data["metadata"]["top_n_lemmas"] = TOP_N_LEMMAS
data["metadata"]["per_n_words"] = PER_WORDS
data["data"] = important_lemmas()
export_json_data("top_lemmas.json", data)

### Question 3

In [14]:

from collections.abc import Callable, Generator
from functools import cache

from eda.language import AttributedWord
from eda.models import MacroRegion, ParticipantLines


def participant_macro_region(participant: Participant) -> MacroRegion:
    conversation = conversations.conversation(participant.conversation_code)
    return conversation.macro_region


def generate_participant_words(lines: ParticipantLines) -> Generator[AttributedWord]:
    for line in lines:
        yield from filter(lambda word: word.is_linguistic, line.normalised_words)

@cache
def participants_dialect_percentages(
    *, rounder: Callable[[int | float, int], int | float] = round_precise
) -> list[int | float]:
    percentages = []

    for participant in participants:
        participant_lines = conversations.participant_lines(participant)
        dialect_words = total_words = 0
        for word in generate_participant_words(participant_lines):
            dialect_words += word.is_dialect(strict=False)
            total_words += 1

        percentange_of_dialect_words = rounder(dialect_words / total_words * 100, 2)
        percentages.append(percentange_of_dialect_words)

    return percentages

def labeled_percentages() -> list[dict[str, Any]]:
    result = {
        key.name: value 
        for key, value in Generation.create_mapping().items()
    }
    
    region_percentages = defaultdict(float)

    dialect_percentages = iter(participants_dialect_percentages())
    for participant in participants:
        region = participant.geographic_origin
        percentage = next(dialect_percentages)
        participant_data = {
            "region": region,
            "dialect_percentage": percentage,
            "macro_region": participant.macro_region.name.lower()
        }
        region_percentages[region] += percentage
        result[participant.generation.name].append(participant_data)
    return result  # type: ignore

data = create_json_data()
data["metadata"]["title"] = "Dialect word percentages"
data["data"] = labeled_percentages()
export_json_data("dialect_percentages.json", data)

In [1]:
from functools import partial
from typing import Optional, cast

import numpy as np
import pandas as pd


def generational_dialect_percentages(top_n: Optional[int] = None) -> pd.DataFrame:
    data = []
    region_percentages = defaultdict(float)

    dialect_percentages = iter(participants_dialect_percentages())
    for participant in participants:
        region = participant.geographic_origin
        percentage = next(dialect_percentages)
        participant_data = {
            "generation": participant.generation.name,
            "dialect_percentage": percentage,
            "region": region
        }
        region_percentages[region] += percentage
        data.append(participant_data)

    top_regions = frozenset(
        region for region, _ in sorted(
            region_percentages.items(), key=lambda pair: pair[1], reverse=True
        )[:top_n]
    )

    for participant_data in data:
        region = participant_data["region"]
        if region not in top_regions:
            participant_data["region"] = "other"

    df = pd.DataFrame(data)
    df = df.groupby(["generation", "region"])["dialect_percentage"].mean().unstack()
    df = df.sort_values(by="generation")
    df = df.replace(np.nan, 0.0)
    return df

def get_region_dialects_df() -> pd.DataFrame:
    region_dialects_df = generational_dialect_percentages()
    region_dialects_df = region_dialects_df[region_dialects_df.columns]
    region_dialects_df = pd.DataFrame(region_dialects_df.apply(partial(round, ndigits=2), axis=1))
    region_dialects_df = region_dialects_df.loc[:, (region_dialects_df != 0).sum() >= 2]
    return region_dialects_df

def generation_grouped_percentages(region_dialects_df: pd.DataFrame) -> dict[str, Any]:
    percentages = {
        generation.name: {}
        for generation in Generation.create_mapping()
    }

    for i, row in region_dialects_df.iterrows():
        values = {
            key: float(cast(np.float64, value)) 
            for key, value in dict(row).items()
        }
        percentages[cast(str, i)] = values

    return percentages

def calculate_regional_deltas(region_dialects_df: pd.DataFrame) -> dict[str, float]:
    def percentage_delta(percentages: list[float]) -> float:
        delta = 0
        prev = percentages[0]
        for percentage in percentages[1:]:
            delta += percentage - prev
            prev = percentage
        return delta
    
    percentages_per_region = defaultdict(list)
    for _, row in region_dialects_df.iterrows():
        for region, percentage in dict(row).items():
            percentages_per_region[region].append(float(percentage))  # type: ignore

    result = {}
    for region, values in percentages_per_region.items():
        values = list(filter(None, values))
        result[region] = round_precise(percentage_delta(values), 2)

    return result

data = create_json_data()
data["metadata"]["title"] = "Changes of percentage of dialect words spoken over generations"
data["data"] = calculate_regional_deltas(get_region_dialects_df())
export_json_data("dialect_delta_percentages.json", data)

NameError: name 'Any' is not defined

In [ ]:
from collections.abc import Generator

from eda.language import AttributedWord
from eda.models import Conversation, Participant
from eda.parsing import ParticipantLines
from eda.utils import round_precise

UNKNOWN_EDUCATION = "N/A"
EDUCATION_RANKINGS = [
    "elem", "dip_tec_prof", "dip_lic", "laurea in corso", "laurea", "med", "phd"
]
GENERATION_ORDER = [
    Generation.BOOMERS, Generation.X, Generation.Y, Generation.Z
]

def participant_dialect_percentage(participant: Participant) -> int | float:
    participant_lines = conversations.participant_lines(participant)
    dialect_words = total_words = 0
    for word in generate_participant_words(participant_lines):
        dialect_words += word.is_dialect(strict=False)
        total_words += 1

    return round_precise(dialect_words / total_words * 100, 2)

def approximate_participant_age(participant: Participant) -> int | float:
    if participant.age_range.is_oldest():
        return participant.age_range.oldest_age
    else:
        return (participant.age_range.youngest_age + participant.age_range.oldest_age) / 2

def conversation_generation(conversation: Conversation) -> Generation:
    # Lower median
    generations = [participant.generation for participant in conversation.participants]
    counts = Counter(generations)
    most_common, count = counts.most_common(1)[0]
    if count > len(generations) / 2:
        return most_common

    generations.sort(key=GENERATION_ORDER.index)
    median_index = (len(generations) - 1) // 2
    return generations[median_index]

def conversation_average_age(conversation: Conversation) -> float:
    age_ranges = list(map(approximate_participant_age, conversation.participants))
    return round_precise(sum(age_ranges) / len(age_ranges))
    
def conversation_educational_background(conversation: Conversation) -> str:
    # Lowest background
    backgrounds = [participant.degree for participant in conversation.participants]
    backgrounds = list(filter(lambda b: b != "N/A", backgrounds))
    counts = Counter(backgrounds)
    most_common, count = counts.most_common(1)[0]
    if count > len(backgrounds) / 2:
        return most_common
    
    return min(backgrounds, key=EDUCATION_RANKINGS.index)
    
def conversation_participant_data(conversation: Conversation) -> dict[str, Any]:
    n_participants = len(conversation.participants)
    dialect_percentages = list(map(participant_dialect_percentage, conversation.participants))

    result = {}
    result["n_participants"] = n_participants
    result["dialect_percentage"] = round_precise(sum(dialect_percentages) / n_participants)

    sort = {}
    sort["average_approximate_age"] = conversation_average_age(conversation)
    sort["generation"] = conversation_generation(conversation).name
    sort["region"] = conversation.macro_region.name.lower()
    sort["educational_background"] = conversation_educational_background(conversation)
    result["sort"] = sort
    return result

conversations.read_all()

data = create_json_data()
data["metadata"]["title"] = "Dialect percentages based on other participant statistics"
data["data"] = {
    conversation.code: conversation_participant_data(conversation) 
    for conversation in sorted(conversations, key=lambda c: c.code)
}
export_json_data("dialect_comparisons.json", data)